In [1]:
# Add mais informações:
    # Quantas vezes o time da casa ganhou do time visitante
    # Quantas vezes o time da casa perdeu do time visitante

📊 Features de desempenho recente
Essas ajudam o modelo a entender como o time está jogando ultimamente:
- `home_team_last_5_win_rate`: percentual de vitórias nos últimos 5 jogos do time da casa.
- `away_team_last_5_win_rate`: idem para o visitante.
- `home_team_goal_avg_last_5`: média de gols marcados pelo time da casa nos últimos 5 jogos.
- `away_team_goal_avg_last_5`: idem para o visitante.
- `home_team_goals_conceded_last_5`: média de gols sofridos nos últimos 5 jogos.
- `home_team_form`: pontos conquistados nos últimos 5 jogos (3 por vitória, 1 por empate, 0 por derrota).


🏠📦 Features de desempenho em casa/fora
Separar o desempenho como mandante e visitante melhora bastante:
- `home_team_home_win_rate`: taxa de vitórias como mandante.
- `away_team_away_win_rate`: taxa de vitórias como visitante.
- `home_team_home_goal_avg`: média de gols marcados em casa.
- `away_team_away_goal_avg`: média de gols marcados fora de casa.


📈 Features históricas de confronto
Conhecido como head-to-head:
- `head_to_head_win_home_team`: quantas vezes o time da casa venceu o visitante historicamente.
- `head_to_head_draws`: quantos empates nos últimos X jogos entre os dois times.
- `head_to_head_goal_diff`: saldo de gols nos últimos confrontos entre eles.


🧠 Features de contexto
Baseado em data, tempo e local:
- `is_weekend`: jogo foi em final de semana?
- `month`: mês do jogo (para ver se tem sazonalidade).
- `days_since_last_match_home`: dias desde o último jogo do time da casa.
- `match_importance`: pode ser calculado se você tiver a rodada ou posição na tabela (por ex. “jogo decisivo”).


⚽️ Features com diferenças entre os times
Você pode criar colunas de diferença entre as estatísticas dos dois times:
- `goal_avg_diff`: diferença entre a média de gols marcados dos dois.
- `win_rate_diff`: diferença entre taxas de vitória.
- `ranking_diff`: se você tiver um ranking ou pontuação.

🔥 Features de momentum e forma recente
Essas capturam o "embalo" dos times:
- `home_team_streak`: número de vitórias consecutivas do time da casa.
- `away_team_streak`: idem para o visitante.
- `home_team_last_match_result`: resultado do último jogo (pode codificar como +1 vitória, 0 empate, -1 derrota).
- `home_team_points_last_3`: soma de pontos conquistados nos últimos 3 jogos (vitória = 3, empate = 1).
- `home_team_score_diff_last_5`: saldo de gols nos últimos 5 jogos.

🧮 Features estatísticas acumuladas
Acumulados por temporada:
- `home_team_total_goals_scored_season`
- `home_team_total_goals_conceded_season`
- `away_team_total_goals_scored_season`
- `away_team_total_goals_conceded_season`
- `home_team_matches_played_season`: total de jogos no campeonato até o momento.
- `home_team_goal_diff_season`: saldo de gols na temporada.

🗓️ Features temporais
- `match_day_of_week`: dia da semana do jogo (Monday, Tuesday, etc).
- `season_phase`: divida a temporada em blocos, tipo começo/meio/final (pode influenciar em jogos decisivos).
- `round_number`: se você tiver como contar qual rodada do campeonato é.

💹 Features baseadas em odds (se tiver)
Se você tiver odds das casas de apostas:
- `home_win_odds`, `draw_odds`, `away_win_odds`
- `implied_prob_home_win`: transforme a odd em probabilidade implícita: `1 / odds`.
- `odds_margin`: margem de lucro da casa de aposta — pode indicar incerteza do mercado.

🧠 Encoding avançado (para nomes de time)
- `home_team_embedding`: usar embeddings para representar times (ex: contar vitórias, gols, jogos... e transformar em vetores).
- `team_strength_score`: crie um "score de força" baseado nos resultados e gols marcados/concedidos.

💡 Algumas ideias bônus
- **Resultado esperado**: diferença esperada de gols com base nas médias recentes.
- **Jogos simultâneos**: quantos jogos acontecem no mesmo dia (às vezes influencia no desempenho).
- **Jogos recentes entre os dois**: tempo desde o último confronto direto.
- **Primeiro jogo da temporada?**: pode ser um booleano (True / False) para indicar isso.
- **Treinador novo?**: se você tiver esses dados, uma feature de “novo técnico” ajuda bastante!



In [2]:
import os
os.chdir('..')  # volta um diretório
os.chdir('..')  # volta um diretório


In [3]:
import pandas as pd
pd.set_option('display.max_columns', 500)

2025-04-13 21:04:51,259 | main | INFO | get_data | get_soccer_data | Dados carregados com sucesso para o país: Brazil
2025-04-13 21:04:51,260 | main | DEBUG | get_data | get_soccer_data | Conexão com o banco de dados fechada.


Conexão SQLite estabelecida.


In [21]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
import pandas as pd
from modelagem.feature_eng.match_analysis import get_storage_ranks, create_main_cols
from modelagem.utils.logs import logger
# Função para encoding dos times
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import json

from modelagem.utils.preprocessing.engine import encode_teams, calculate_match_points, get_feature_columns



# Definindo diretórios base
MODEL_DIR = os.path.join('database', 'models')
LOG_DIR = os.path.join('logs')
FT_DIR = Path("features")
LOG_DIR = Path("logs")


from modelagem.utils.get_data import get_soccer_data
df = get_soccer_data()

df = df.copy() 


# # Criando colunas iniciais
# df['match_name'] = df['home_team'] + ' - ' + df['away_team']
# df['datetime'] = pd.to_datetime(df['datetime'])

# # Selecionando colunas importantes
# df = df[["season", "datetime", "home_team", "away_team", 
#             "home_score", "away_score", "result", "match_name"]]

# # Convertendo colunas para inteiro
# to_int = ['season', 'home_score', 'away_score']
# df[to_int] = df[to_int].apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

# # Encoding dos times
# df = encode_teams(df,
#                     path_team_mapping=os.path.join(MODEL_DIR, "team_mapping.json"))

# # Calculando pontos e resultado das partidas
# logger.debug("Iniciando o cálculo dos pontos e resultados das partidas.")
# df = calculate_match_points(df)

# # Reordenando colunas
# cols_order = ['season','datetime', 'result', 'match_name', 'home_team', 'away_team',
#                 'home_team_encoder', 'away_team_encoder', 'winner', 'home_score', 
#                 'away_score', 'h_match_points', 'a_match_points']
# df = df[cols_order]

# # Feature Engineering
# logger.debug("Iniciando o feature engineering.")
# df_storage_ranks = get_storage_ranks(df)







# logger.debug("Iniciando o feature engineering do time da casa.")
# ht_cols = [f'ht{col}' for col in get_feature_columns()]
# at_cols = [f'at{col}' for col in get_feature_columns()]

# logger.debug("Criando create_main_cols")
# df[ht_cols] = df.apply(lambda x: create_main_cols(x, x.home_team, df, df_storage_ranks), axis=1, result_type='expand')
# df[at_cols] = df.apply(lambda x: create_main_cols(x, x.away_team, df, df_storage_ranks), axis=1, result_type='expand')

# # Removendo colunas desnecessárias
# df.drop(columns=['match_name', 'datetime', 'home_team', 'away_team', 
#                     'home_score', 'away_score', 'h_match_points', 'a_match_points'], inplace=True)

# df.fillna(-33, inplace=True)  # Preenchendo valores ausentes

2025-04-13 21:19:44,281 | main | INFO | get_data | get_soccer_data | Dados carregados com sucesso para o país: Brazil
2025-04-13 21:19:44,283 | main | DEBUG | get_data | get_soccer_data | Conexão com o banco de dados fechada.


Conexão SQLite estabelecida.


In [22]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y = encoder.fit_transform(df['result'])  # 'H' → 0, 'D' → 1, 'A' → 2

df["map_result"] = y



In [23]:
# !pip install jinja2

In [24]:
# from modelagem.feature_eng.match_analysis import get_storage_ranks
from modelagem.utils.create_features import (
    get_recent_performance,
    add_home_away_stats,
    add_head_to_head_features,
    add_context_features,
    update_team_ranking,
    check_and_create_features,
    add_diff_features,
    calc_momentum_features,
    calc_momentum_features,
    add_season_stats,
    add_temporal_features,
)

In [ ]:

df_with_features = df.copy()
df_with_features = get_recent_performance(df_with_features)
df_with_features = add_home_away_stats(df_with_features)
df_with_features = add_head_to_head_features(df_with_features)
df_with_features = add_context_features(df_with_features)
df_with_features = update_team_ranking(df_with_features)
df_with_features = check_and_create_features(df_with_features)
df_with_features = add_diff_features(df_with_features)
df_with_features = calc_momentum_features(df_with_features, is_home=False, team_col='home_team')
df_with_features = calc_momentum_features(df_with_features, is_home=False, team_col='away_team')
df_with_features = add_season_stats(df_with_features)
df_with_features = add_temporal_features(df_with_features)

ℹ️ Colunas de ranking não encontradas (pulando ranking_diff)


In [13]:
from modelagem.utils.preprocessing.engine import calculate_match_points

calculate_match_points(df_with_features)

,season,datetime,result,match_name,home_team,away_team,home_team_encoder,away_team_encoder,winner,home_score,away_score,h_match_points,a_match_points,map_result,home_team_last_5_win_rate,away_team_last_5_win_rate,home_team_goal_avg_last_5,away_team_goal_avg_last_5,home_team_goals_conceded_last_5,away_team_goals_conceded_last_5,home_team_form,away_team_form,home_team_home_win_rate,away_team_away_win_rate,home_team_home_goal_avg,away_team_away_goal_avg,head_to_head_win_home_team,head_to_head_draws,head_to_head_goal_diff,is_weekend,month,days_since_last_match_home,match_importance,home_team_ranking,away_team_ranking,goal_avg_diff,win_rate_diff,ranking_diff,away_team_streak,away_team_last_match_result,away_team_points_last_3,away_team_score_diff_last_5,home_team_total_goals_scored_season,home_team_total_goals_conceded_season,home_team_matches_played_season,home_team_goal_diff_season,away_team_total_goals_scored_season,away_team_total_goals_conceded_season,match_day_of_week,season_phase
0,2012,2012-05-19 22:30:00,D,Palmeiras - Portuguesa,Palmeiras,Portuguesa,27,30,0,1,1,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,True,5,NaN,0,1,1,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,Saturday,mid
1,2012,2012-05-19 22:30:00,D,Sport Recife - Flamengo RJ,Sport Recife,Flamengo RJ,34,17,0,1,1,1,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,True,5,NaN,0,1,1,0.0,0.0,0,0,0,0,0,0,0,0,0,0,0,Saturday,mid
2,2012,2012-05-20 01:00:00,H,Figueirense - Nautico,Figueirense,Nautico,16,26,2,2,1,3,0,2,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,True,5,NaN,0,3,0,0.0,0.0,3,0,0,0,0,0,0,0,0,0,0,Sunday,mid
3,2012,2012-05-20 20:00:00,H,Botafogo RJ - Sao Paulo,Botafogo RJ,Sao Paulo,6,33,2,4,2,3,0,2,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,True,5,NaN,0,3,0,0.0,0.0,3,0,0,0,0,0,0,0,0,0,0,Sunday,mid
4,2012,2012-05-20 20:00:00,A,Corinthians - Fluminense,Corinthians,Fluminense,11,18,1,0,1,0,3,0,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,0.0,0.0,0,0,0,True,5,NaN,0,0,3,0.0,0.0,-3,0,0,0,0,0,0,0,0,0,0,Sunday,mid
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4956,2025,2025-04-06 22:30:00,A,Sport Recife - Palmeiras,Sport Recife,Palmeiras,34,27,1,1,2,0,3,0,0.2,0.4,0.6,0.8,0.8,1.0,6,7,0.4,0.8,1.4,2.0,0,1,-4,True,4,1213.0,0,1,4,-0.2,-0.2,-3,3,1,9,3,0,0,1,0,0,0,Sunday,mid
4957,2025,2025-04-06 22:30:00,A,Vitoria - Flamengo RJ,Vitoria,Flamengo RJ,36,17,1,1,2,0,3,0,0.2,0.4,1.2,1.8,1.2,1.0,6,9,0.6,0.6,1.4,1.4,0,2,-3,True,4,122.0,0,0,4,-0.6,-0.2,-4,1,1,7,3,0,2,1,-2,1,1,Sunday,mid
4954,2025,2025-04-06 22:30:00,H,Internacional - Cruzeiro,Internacional,Cruzeiro,22,14,2,3,0,3,0,2,0.2,0.4,1.4,1.2,1.8,1.0,4,8,0.6,0.2,1.8,0.6,2,3,2,True,4,122.0,0,4,3,0.2,-0.2,1,1,1,4,-2,1,1,1,0,2,1,Sunday,mid
4955,2025,2025-04-06 22:30:00,D,Mirassol - Fortaleza,Mirassol,Fortaleza,25,19,0,1,1,1,1,1,0.0,0.4,1.0,1.2,2.0,1.0,0,7,0.0,0.2,0.0,1.6,0,0,0,True,4,NaN,0,1,4,-0.2,-0.4,-3,0,-1,1,-1,1,2,1,-1,2,0,Sunday,mid


In [10]:
drop_columns = df_with_features.select_dtypes(include=['object']).columns

result = df_with_features.drop(columns=drop_columns).corr()

result.style.background_gradient(cmap='coolwarm', axis=None)

,season,datetime,home_team_encoder,away_team_encoder,winner,home_score,away_score,h_match_points,a_match_points,map_result,home_team_last_5_win_rate,away_team_last_5_win_rate,home_team_goal_avg_last_5,away_team_goal_avg_last_5,home_team_goals_conceded_last_5,away_team_goals_conceded_last_5,home_team_form,away_team_form,home_team_home_win_rate,away_team_away_win_rate,home_team_home_goal_avg,away_team_away_goal_avg,head_to_head_win_home_team,head_to_head_draws,head_to_head_goal_diff,is_weekend,month,days_since_last_match_home,match_importance,home_team_ranking,away_team_ranking,goal_avg_diff,win_rate_diff,ranking_diff,away_team_streak,away_team_last_match_result,away_team_points_last_3,away_team_score_diff_last_5,home_team_total_goals_scored_season,home_team_total_goals_conceded_season,home_team_matches_played_season,home_team_goal_diff_season,away_team_total_goals_scored_season,away_team_total_goals_conceded_season
season,1.000000,0.998736,-0.121561,-0.121646,-0.024378,-0.023860,0.028169,-0.031633,0.029359,-0.031008,0.010667,0.002867,0.019719,0.000114,0.003301,0.018144,0.026846,0.014366,-0.013407,0.074497,-0.029078,0.070359,0.290573,0.254631,0.005116,-0.049970,-0.086985,0.062228,0.012583,-0.015341,-0.012565,0.013830,0.005531,-0.004020,0.042959,0.031123,0.068476,0.045006,-0.011713,-0.013653,-0.012167,0.002548,-0.012766,-0.010265
datetime,0.998736,1.000000,-0.121601,-0.121583,-0.024032,-0.023398,0.028566,-0.031854,0.029834,-0.031357,0.012072,0.003982,0.020362,0.000270,0.003218,0.019030,0.028602,0.015576,-0.012164,0.077240,-0.028535,0.071252,0.290592,0.257790,0.002448,-0.054350,-0.052122,0.051501,0.047924,0.025500,0.028296,0.014172,0.005732,-0.003947,0.044113,0.031284,0.070728,0.044889,0.029976,0.028462,0.034294,0.002663,0.028837,0.031934
home_team_encoder,-0.121561,-0.121601,1.000000,-0.029378,0.016662,0.025799,0.000540,0.022443,-0.021159,0.022161,0.033244,0.012186,0.032275,0.002862,0.002293,-0.018936,0.031007,0.008662,0.043311,0.000541,0.051686,-0.012021,0.013146,0.005297,0.021632,-0.001561,0.002298,-0.001895,-0.010947,0.017748,-0.000913,0.020721,0.014912,0.026811,0.002458,0.012013,0.006950,0.008361,0.016869,-0.004034,-0.002322,0.029924,0.003239,-0.001718
away_team_encoder,-0.121646,-0.121583,-0.029378,1.000000,-0.005808,-0.000579,0.004099,-0.011781,0.012636,-0.012386,-0.015768,0.039119,-0.013767,0.037179,-0.020992,-0.004682,-0.011707,0.040174,0.008618,0.022259,0.001883,0.009724,-0.033043,0.006130,-0.023510,0.013841,0.007090,0.009445,-0.006212,0.002792,0.021287,-0.036325,-0.039198,-0.026522,0.005842,0.012617,0.020074,0.025472,0.002519,0.000903,0.000399,0.002335,0.020669,-0.003639
winner,-0.024378,-0.024032,0.016662,-0.005808,1.000000,0.512643,-0.265380,0.749992,-0.476455,0.627070,0.049742,-0.079870,0.064453,-0.072938,-0.025965,0.047899,0.057045,-0.077090,0.085941,-0.089187,0.072164,-0.059485,0.061644,-0.002185,0.079890,-0.014434,0.007367,-0.000232,0.007857,0.117093,-0.069406,0.097679,0.092504,0.267803,-0.048679,-0.025766,-0.081344,-0.086327,0.058567,-0.015527,0.015621,0.106046,-0.027409,0.041194
home_score,-0.023860,-0.023398,0.025799,-0.000579,0.512643,1.000000,0.025397,0.628988,-0.569255,0.609414,0.092198,-0.073993,0.087944,-0.047097,-0.068903,0.081288,0.094924,-0.080264,0.125543,-0.064735,0.121820,-0.035392,0.065518,0.028933,0.110073,-0.016357,0.002417,-0.001609,0.007255,0.134555,-0.082067,0.095755,0.118479,0.311054,-0.011823,-0.013706,-0.050587,-0.066548,0.076772,-0.039525,0.013482,0.166188,-0.022523,0.062084
away_score,0.028169,0.028566,0.000540,0.004099,-0.265380,0.025397,1.000000,-0.575508,0.626854,-0.609773,-0.063289,0.071093,-0.061507,0.100544,0.052098,-0.028183,-0.068688,0.076748,-0.070593,0.068574,-0.050455,0.101818,-0.042686,0.005746,-0.068821,-0.025271,-0.009822,0.003856,0.011520,-0.080259,0.089180,-0.115362,-0.095854,-0.243241,0.040659,0.051419,0.071695,0.090283,-0.022313,0.031141,-0.006682,-0.076162,0.054927,-0.022961
h_match_points,-0.031633,-0.031854,0.022443,-0.011781,0.749992,0.628988,-0.575508,1.000000,-0.938

In [11]:
result["map_result"].reset_index()

,index,map_result
0,season,-0.031008
1,datetime,-0.031357
2,home_team_encoder,0.022161
3,away_team_encoder,-0.012386
4,winner,0.627070
5,home_score,0.609414
6,away_score,-0.609773
7,h_match_points,0.985540
8,a_match_points,-0.983634
9,map_result,1.000000
